In [1]:
from transformers import pipeline
import torch

In [2]:
generate_text = pipeline(model="databricks/dolly-v2-3b", torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto")

In [3]:
class journallingClass:
    def __init__(self, journalType, title, category, dateCreated, mood, journalClass):
        self.journalType = journalType
        self.journalTitle = title
        self.journalCategory = category 
        self.dateCreated = dateCreated
        self.mood = mood
        self.journalClass = journalClass
    
    def getJournalClass(self):
        return self.journalClass
        
        
class journalEntryClass:
    def __init__(self, userEntry, harmIntent = 0, modelResponse = []):
        self.userEntry = userEntry 
        self.harmIntent = harmIntent
        self.modelResponse = modelResponse
#         self.safetyResponse = safetyResponse
    def printEntry(self):
        print(f"entry: {self.userEntry}, harmful intent: {self.harmIntent}, model response: {self.modelResponse}")

        
    def updateHarmIntent(self, harmIntent):
        self.harmIntent = harmIntent
    
    def addModelResponse(self, response):
        self.modelResponse.append(response)
        
class casualJournal(journallingClass):
    
    def __init__(self, entries = []):
        self.entries = []
        
    def createNewEntry(self, userEntry,  harmIntent = 0, modelResponse = []):
        self.entries.append(journalEntryClass(userEntry, harmIntent, modelResponse))
        return self.entries[-1]
        
    def printJournal(self):
        for entry in self.entries:
            entry.printEntry()
    
    
        
    
class catharticJournal(journallingClass):
    def __init__(self, stage = 0, entries = {}):
        self.stage = stage # default stage 0
        self.entries = entries
        
    
    def createStage(self, journalEntry):
        self.entries[f"stage{self.stage}"] = journalEntry
    
    def updateStage(self):
        if self.stage >=5:
            return 
        self.stage += 1
        
    def createNewEntry(self, userEntry,  harmIntent = 0, modelResponse = []):
        self.updateStage()
        self.entries[f"stage{self.stage}"] = journalEntryClass(userEntry, harmIntent, modelResponse)
        return self.entries[f"stage{self.stage}"]
    
    def printJournal(self):
        for stage in self.entries.keys():
            print(stage)
            self.entries[stage].printEntry()


In [4]:
# def generate_text(userEntry):
#     return "<Model's empathetic Response>"

def checkHarmfulIntent(journalEntry):
    # implement later
    journalEntry.updateHarmIntent(0)
    # If this is harmful update the harmful intent to be set to 1
    return False 

def buildHarmresponse():
    response = "As an AI language model, I don't have feelings or emotions, but I am designed to provide helpful and supportive responses to you. However, it seems like you are going through a challenging time and dealing with issues that may benefit from professional help, I would recommend that you consider speaking with a licensed therapist or counselor."
    return response

def generateCatharticResponse(journalEntry, journal):
    if journal.getJournalClass().stage == 5:
        prompt = f"Pretend to be my therapist and help me process my emotions. Do not try to provide solutions and instead provide empathy while keeping your response under 100 words. Can you give me other ways of thinking about the situation? This is the given scenario: {journalEntry.userEntry}"
    else:
        prompt = f"Pretend to be my therapist and help me process my emotions. Do not try to provide solutions and instead provide empathy while keeping your response under 100 words. This is the given scenario: {journalEntry.userEntry}"
    
    print(f"Prompt: {prompt}")
    response = generate_text(prompt)
    return response[0]

def generateCasualResponse(journalEntry):
    prompt = f"Pretend to be my therapist and help me process my emotions. Do not try to provide solutions and instead provide empathy while keeping your response under 100 words. This is the given scenario: {journalEntry.userEntry}"
    print(f"Prompt: {prompt}")
    res = generate_text(prompt)
    return res[0]

def promptSafetyDetector(journalEntry, safety_counter):
    return True

def generatePrompt(typeJ, journalEntry, journal):
    safety_counter = 0
    safety_threshold = 5
    
    while(safety_counter<safety_threshold):
        if typeJ == 1:
            response = generateCatharticResponse(journalEntry, journal)
        else:
            response = generateCasualResponse(journalEntry)
        # check safety
        safe = promptSafetyDetector(journalEntry, safety_counter)
        if safe:
            break
        safety_counter += 1
        
    if safety_counter == safety_threshold:
        response = "<default response - model response was unsafe.>"
    
    # add model response to current entry object
    journalEntry.addModelResponse(response)
    
    return response

In [5]:
def main(response):
    
    if response["type"] == 0: # get type check from backend
        journalChild = casualJournal()
    else:
        journalChild = catharticJournal()
        
    # pull journalType, title, category, dateCreated from backend
    journal = journallingClass(response["type"], response["title"], response["category"], response["date"], response["mood"], journalChild)
    
    # pull journal entries, loop through and load 
    for entry, harmfulIntent, modelResponse in zip(response["entry"], response["harmfulIntent"], response["modelResponse"]):
        journal.getJournalClass().createNewEntry(entry, harmfulIntent, modelResponse)
    
    # get latest entry
    currentEntry = journal.getJournalClass().createNewEntry(response["entry"][-1])
    
    currentEntry.printEntry()
    
    
    harmIntent = checkHarmfulIntent(currentEntry)
    
    if harmIntent: 
        modelResponse = buildHarmresponse() 
        currentEntry.addModelResponse(modelResponse)
        result["modelResponse"].append(modelResponse)
        result["harmfulIntent"].append(1) # confirmm if 0/1 or true/false
        return result
    
    modelResponse = generatePrompt(response["type"], currentEntry, journal)
    
#     currentEntry.printEntry()
    journal.getJournalClass().printJournal()
    
    
    result = response
    result["modelResponse"].append(modelResponse)
    result["harmfulIntent"].append(0)
    return result
    

In [6]:
response = {
    "entry": ["I'm really stressed and overwhelmed with school right now. There are a lot of deliverables coming up and not a lot of time. I feel anxious and tired. There's so much to do and I don't feel like doing any of it because I feel overwhelmed. I have three this due this weekend and 2 big projects due next week."],
    "harmfulIntent": [],
    "modelResponse": [],
    "type": 0, # assume 0 cathartic,
    "title": "exam stress", 
    "userID": 8940823, 
    "date": "14/12/2023", 
    "category": "cat died",
    "mood": "mood"
}
result = main(response)

entry: I'm really stressed and overwhelmed with school right now. There are a lot of deliverables coming up and not a lot of time. I feel anxious and tired. There's so much to do and I don't feel like doing any of it because I feel overwhelmed. I have three this due this weekend and 2 big projects due next week., harmful intent: 0, model response: []
Prompt: Pretend to be my therapist and help me process my emotions. Do not try to provide solutions and instead provide empathy while keeping your response under 100 words. This is the given scenario: I'm really stressed and overwhelmed with school right now. There are a lot of deliverables coming up and not a lot of time. I feel anxious and tired. There's so much to do and I don't feel like doing any of it because I feel overwhelmed. I have three this due this weekend and 2 big projects due next week.
entry: I'm really stressed and overwhelmed with school right now. There are a lot of deliverables coming up and not a lot of time. I feel a

In [7]:
response

{'entry': ["I'm really stressed and overwhelmed with school right now. There are a lot of deliverables coming up and not a lot of time. I feel anxious and tired. There's so much to do and I don't feel like doing any of it because I feel overwhelmed. I have three this due this weekend and 2 big projects due next week."],
 'harmfulIntent': [0],
 'modelResponse': [{'generated_text': "Wow, that's stressful. It's hard to feel overwhelmed with so many things and not being in control of any of it. I'm not sure how I would feel in that situation, but I can understand why you would feel that way. My recommendation is to focus on things that you are in control of and are giving you comfort. You know how to handle your responsibilities and projects so you can put that ahead of everything else."}],
 'type': 0,
 'title': 'exam stress',
 'userID': 8940823,
 'date': '14/12/2023',
 'category': 'cat died',
 'mood': 'mood'}

In [8]:
result

{'entry': ["I'm really stressed and overwhelmed with school right now. There are a lot of deliverables coming up and not a lot of time. I feel anxious and tired. There's so much to do and I don't feel like doing any of it because I feel overwhelmed. I have three this due this weekend and 2 big projects due next week."],
 'harmfulIntent': [0],
 'modelResponse': [{'generated_text': "Wow, that's stressful. It's hard to feel overwhelmed with so many things and not being in control of any of it. I'm not sure how I would feel in that situation, but I can understand why you would feel that way. My recommendation is to focus on things that you are in control of and are giving you comfort. You know how to handle your responsibilities and projects so you can put that ahead of everything else."}],
 'type': 0,
 'title': 'exam stress',
 'userID': 8940823,
 'date': '14/12/2023',
 'category': 'cat died',
 'mood': 'mood'}